#### Libraries

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

# --- Settings ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Data

In [2]:
launches_file_path = '../data/transformed/launch/usa_launches.csv'
wx_decisions_file_path = '../data/raw/launch/launch_weather_stats.csv'
stats = pd.read_csv('../data/raw/launch/clean_launch_stats.csv')
tap_lab_data = pd.read_csv('../data/raw/launch/launch_data_from_tap_lab.csv')

In [3]:
# --- Files ---
launches_file_path = '../data/transformed/launch/usa_launches.csv'
wx_decisions_file_path = '../data/raw/launch/launch_weather_stats.csv'
stats = pd.read_csv('../data/raw/launch/clean_launch_stats.csv')

# --- Launch Data ---
launches = pd.read_csv(launches_file_path,
                       parse_dates=['date'],
                       dtype={col: str for col in pd.read_csv(launches_file_path).columns if col != 'date'},
                       )
launches = launches.rename(columns={'date': 'LIFTOFF_DATETIME'})
launches['LIFTOFF_DATE'] = launches['LIFTOFF_DATETIME'].dt.date
launches.columns = launches.columns.str.upper()

# --- LCC Launch Stat Data ---
stats.columns = stats.columns.str.upper()
stats = stats.drop(columns=['GOES_IMG_PATH', 'SHEAR_IMG_PATH', 'WARNIGN_IMG_PATH', 'SBCAPE_CIN_IMG_PATH'])

# --- Weather Decisions Data ---
wx_decisions = pd.read_csv(wx_decisions_file_path,
                           parse_dates=['Date'],
                           dtype={'Launch_Vehicle': str, 'Payload': str, 'Remarks': str},
                           na_values=[' ', 'NaN', 'NA'])
wx_decisions.columns = wx_decisions.columns.str.upper()
wx_decisions['DATE'] = pd.to_datetime(wx_decisions['DATE'], errors='coerce')

bool_columns = ['COUNTDOWN', 'LAUNCHED', 'NON_WX_SCRUB', 'WX_SCRUB', 
                'NON_WX_DELAY', 'WX DELAY', 'LCC_SCRUB_DELAY', 'USER_WX_SCRUB_DELAY']

# Replace NaN with 0 and convert to int
for col in bool_columns:
    wx_decisions[col] = pd.to_numeric(wx_decisions[col], errors='coerce').fillna(0).astype(int)

In [4]:
launches.sample(5)

,NAME,STATUS,PROVIDER,ROCKET,MISSION,LIFTOFF_DATETIME,LOCATION,PAD,LIFTOFF_DATE
753,Delta 2914 | SMS 1,Launch Failure,United States Air Force,Delta 2914,SMS 1,1974-05-17 09:31:00,"Cape Canaveral, FL, USA",Space Launch Complex 17B,1974-05-17
316,Atlas SLV-3 Agena D | KH-7 15,Launch Successful,United States Air Force,Atlas SLV-3 Agena D,KH-7 15,1965-01-23 20:06:00,"Vandenberg SFB, CA, USA",Space Launch Complex 4W,1965-01-23
1740,Falcon 9 Block 5 | Starlink 4,Launch Successful,SpaceX,Falcon 9,Starlink 4,2020-02-17 15:05:00,"Cape Canaveral, FL, USA",Space Launch Complex 40,2020-02-17
1758,Starship SN6 | 150m Hop,Launch Successful,SpaceX,Starship Prototype,SN6 150m Hop,2020-09-03 17:48:00,"SpaceX Starbase, TX, USA",Suborbital Pad A,2020-09-03
708,Scout B-1 | Triad 1,Launch Successful,United States Air Force,Scout B-1,Triad 1,1972-09-02 17:50:00,"Vandenberg SFB, CA, USA",Space Launch Complex 5,1972-09-02


In [5]:
# Display the launches dataframe as markdown
print(launches.drop(columns=['LIFTOFF_DATE', 'ROCKET']).sample(3).transpose().to_markdown(tablefmt='pipe'))

|                  | 548                     | 486                            | 1233                                     |
|:-----------------|:------------------------|:-------------------------------|:-----------------------------------------|
| NAME             | Atlas F | OV1-13 & 14   | Thor SLV-2A Agena D | KH-4A 40 | Space Shuttle Columbia / OV-102 | STS-80 |
| STATUS           | Launch Successful       | Launch Successful              | Launch Successful                        |
| PROVIDER         | United States Air Force | United States Air Force        | United Space Alliance                    |
| MISSION          | OV1-13 & 14             | KH-4A 40                       | STS-80                                   |
| LIFTOFF_DATETIME | 1968-04-06 09:59:00     | 1967-03-30 18:54:00            | 1996-11-19 19:55:00                      |
| LOCATION         | Vandenberg SFB, CA, USA | Vandenberg SFB, CA, USA        | Kennedy Space Center, FL, USA            |
| PAD           

In [6]:
wx_decisions.sample(5)

,LAUNCH_VEHICLE,PAYLOAD,DATE,COUNTDOWN,LAUNCHED,NON_WX_SCRUB,WX_SCRUB,NON_WX_DELAY,WX DELAY,LCC_SCRUB_DELAY,USER_WX_SCRUB_DELAY,REMARKS
60,DELTA II-204,NaN,1991-04-14,1,1,0,0,0,1,0,1,LOADS
569,ATLAS V,OA-7,2017-04-18,1,1,0,0,0,0,0,0,NaN
148,TITAN IV,NaN,1994-12-22,1,1,0,0,0,0,0,0,NaN
207,DELTA II,NaN,1997-01-16,1,0,0,1,0,0,0,1,LOADS
552,ATLAS V,NROL-61,2016-07-28,1,1,0,0,0,0,0,0,NaN


In [7]:
print(wx_decisions[wx_decisions['REMARKS'].notna()].sample(3).transpose().to_markdown(tablefmt='pipe'))

|                     | 156                      | 675                    | 349                 |
|:--------------------|:-------------------------|:-----------------------|:--------------------|
| LAUNCH_VEHICLE      | AC 77                    | Falcon 9               | DELTA II            |
| PAYLOAD             | nan                      | Starlink-12            | nan                 |
| DATE                | 1995-05-19 00:00:00      | 2020-10-01 00:00:00    | 2003-06-09 00:00:00 |
| COUNTDOWN           | 1                        | 1                      | 1                   |
| LAUNCHED            | 0                        | 0                      | 0                   |
| NON_WX_SCRUB        | 1                        | 1                      | 0                   |
| WX_SCRUB            | 0                        | 0                      | 1                   |
| NON_WX_DELAY        | 0                        | 0                      | 0                   |
| WX DELAY          

In [8]:
stats.sample(5)

,LAUNCH_VEHICLE,PAYLOAD,DATE,START_LCC_EVAL,END_LCC_EVAL,TOTAL_EVAL_TIME,LCC_VIOLATION_COUNT,LIGHTNING_RULE_VIOLATED,LIGHTNING_RULE_VIOLATION_COUNT,LIGHTNING_RULE_VIOLATION_DURATION,CUMULUS_RULE_VIOLATED,CUMULUS_RULE_VIOLATION_COUNT,CUMULUS_RULE_VIOLATION_DURATION,ANVIL_2009_RULE_VIOLATED,ANVIL_2009_RULE_VIOLATION_COUNT,ANVIL_2009_RULE_VIOLATION_DURATION,ATTACHED_ANVIL_RULE_VIOLATED,ATTACHED_ANVIL_RULE_VIOLATION_COUNT,ATTACHED_ANVIL_RULE_VIOLATION_DURATION,DETACHED_ANVIL_RULE_VIOLATED,DETACHED_ANVIL_RULE_VIOLATION_COUNT,DETACHED_ANVIL_RULE_VIOLATION_DURATION,DEBRIS_RULE_VIOLATED,DEBRIS_RULE_VIOLATION_COUNT,DEBRIS_RULE_VIOLATION_DURATION,DISTURBED_RULE_VIOLATED,DISTURBED_RULE_VIOLATION_COUNT,DISTURBED_RULE_VIOLATION_DURATION,THICK_RULE_VIOLATED,THICK_RULE_VIOLATION_COUNT,THICK_RULE_VIOLATION_DURATION,SMOKE_RULE_VIOLATED,SMOKE_RULE_VIOLATION_COUNT,SMOKE_RULE_VIOLATION_DURATION,FIELD_MILL_RULE_VIOLATED,FIELD_MILL_RULE_VIOLATION_COUNT,FIELD_MILL_RULE_VIOLATION_DURATION,GOOD_SENSE_RULE_VIOLATED,GOOD_SENSE_RULE_VIOLATION_COUNT,GOOD_SENSE_RULE_VIOLATION_DURATION,USER_WX_CONSTRAINT_RULE_VIOLATED,USER_WX_CONSTRAINT_RULE_VIOLATION_COUNT,USER_WX_CONSTRAINT_RULE_VIOLATION_DURATION
106,FALCON 9,STARLINK-17,2021-03-01,2021-02-28 23:37:00,2021-03-01 01:35:00,0 days 01:58:00,0,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00
76,FALCON 9,STARLINK-12,2020-09-17,2020-09-17 16:19:00,2020-09-17 17:38:00,0 days 01:19:00,0,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00
112,FALCON 9,CREW-2,2021-04-23,2021-04-23 07:49:00,2021-04-23 09:49:00,0 days 02:00:00,0,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00
123,FALCON 9,TRANSPORTER-2,2021-06-29,2021-06-29 16:56:00,2021-06-29 18:55:00,0 days 01:59:00,1,False,0,0 days 00:00:00,True,1,0 days 00:55:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,True,1,0 days 00:27:00,False,0,0 days,False,0,0 days 00:00:00
231,FALCON 9,INTELSAT-40E,2023-04-07,2023-04-07 02:30:00,2023-04-07 04:30:00,0 days 02:00:00,0,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00


##### Weather Data Wrangling

In [9]:
# Filter wx_decisions to show only those with the same DATE
duplicate_dates_wx = wx_decisions[wx_decisions['DATE'].duplicated(keep=False)]
display(duplicate_dates_wx)


,LAUNCH_VEHICLE,PAYLOAD,DATE,COUNTDOWN,LAUNCHED,NON_WX_SCRUB,WX_SCRUB,NON_WX_DELAY,WX DELAY,LCC_SCRUB_DELAY,USER_WX_SCRUB_DELAY,REMARKS
772,Atlas V,SBIRS GEO 6,2022-08-04,1,1,0,0,0,0,0,0,NaN
773,Falcon 9,KPLO,2022-08-04,1,1,0,0,0,0,0,0,NaN
798,Falcon 9,Eutelsat-10B,2022-11-22,1,0,1,0,0,0,0,0,Payload comm issues
799,Falcon 9,CRS-26,2022-11-22,1,0,0,1,0,0,1,0,NaN
816,Falcon 9,Starlink 6-1,2023-02-27,1,1,0,0,0,0,0,0,NaN
817,Falcon 9,Crew-6,2023-02-27,1,0,1,0,0,0,0,0,NaN


##### Launch Data

In [10]:
# --- Launch Data ---

# Filter launches to only Cape Canaveral, FL, USA and date range
launches = launches[
    (launches['LOCATION'] == 'Cape Canaveral, FL, USA') &
    (launches['LIFTOFF_DATE'] >= pd.to_datetime('1987-11-29').date()) &
    (launches['LIFTOFF_DATE'] <= pd.to_datetime('2023-04-19').date())
]

# Display the shape of the filtered dataset
print(f"Shape of filtered launches dataset: {launches.shape}")

# Display the first few rows to verify the filter
display(launches.head())

Shape of filtered launches dataset: (472, 9)


,NAME,STATUS,PROVIDER,ROCKET,MISSION,LIFTOFF_DATETIME,LOCATION,PAD,LIFTOFF_DATE
1032,Titan 34D Transtage | DSP F13,Launch Successful,United States Air Force,Titan 34D Transtage,DSP F13,1987-11-29 03:27:00,"Cape Canaveral, FL, USA",Space Launch Complex 40,1987-11-29
1034,Delta 3910 | TVE,Launch Successful,United States Air Force,Delta 3910,TVE,1988-02-08 22:07:00,"Cape Canaveral, FL, USA",Space Launch Complex 17B,1988-02-08
1038,Titan 34D Transtage | Mercury 12,Launch Successful,United States Air Force,Titan 34D Transtage,Mercury 12,1988-09-02 12:05:00,"Cape Canaveral, FL, USA",Space Launch Complex 40,1988-09-02
1044,Delta 6925 | GPS-2 1,Launch Successful,United States Air Force,Delta II,GPS-2 1,1989-02-14 18:30:00,"Cape Canaveral, FL, USA",Space Launch Complex 17A,1989-02-14
1046,Delta 3920-8 | Delta Star,Launch Successful,United States Air Force,Delta 3920-8,Delta Star,1989-03-24 21:50:00,"Cape Canaveral, FL, USA",Space Launch Complex 17B,1989-03-24


In [11]:
# Filter launches to show only those with the same LIFTOFF_DATE
duplicate_launches = launches[launches['LIFTOFF_DATE'].duplicated(keep=False)]
display(duplicate_launches)

,NAME,STATUS,PROVIDER,ROCKET,MISSION,LIFTOFF_DATETIME,LOCATION,PAD,LIFTOFF_DATE
1871,Atlas V 421 | SBIRS GEO-6,Launch Successful,United Launch Alliance,Atlas V 421,SBIRS GEO-6,2022-08-04 10:29:00,"Cape Canaveral, FL, USA",Space Launch Complex 41,2022-08-04
1873,Falcon 9 Block 5 | Danuri (KPLO - Korean Pathf...,Launch Successful,SpaceX,Falcon 9,Danuri (KPLO - Korean Pathfinder Lunar Orbiter),2022-08-04 23:08:00,"Cape Canaveral, FL, USA",Space Launch Complex 40,2022-08-04


##### Launch Stats Wrangling

In [12]:
duplicate_dates = stats[stats['DATE'].duplicated(keep=False)]
display(duplicate_dates)

,LAUNCH_VEHICLE,PAYLOAD,DATE,START_LCC_EVAL,END_LCC_EVAL,TOTAL_EVAL_TIME,LCC_VIOLATION_COUNT,LIGHTNING_RULE_VIOLATED,LIGHTNING_RULE_VIOLATION_COUNT,LIGHTNING_RULE_VIOLATION_DURATION,CUMULUS_RULE_VIOLATED,CUMULUS_RULE_VIOLATION_COUNT,CUMULUS_RULE_VIOLATION_DURATION,ANVIL_2009_RULE_VIOLATED,ANVIL_2009_RULE_VIOLATION_COUNT,ANVIL_2009_RULE_VIOLATION_DURATION,ATTACHED_ANVIL_RULE_VIOLATED,ATTACHED_ANVIL_RULE_VIOLATION_COUNT,ATTACHED_ANVIL_RULE_VIOLATION_DURATION,DETACHED_ANVIL_RULE_VIOLATED,DETACHED_ANVIL_RULE_VIOLATION_COUNT,DETACHED_ANVIL_RULE_VIOLATION_DURATION,DEBRIS_RULE_VIOLATED,DEBRIS_RULE_VIOLATION_COUNT,DEBRIS_RULE_VIOLATION_DURATION,DISTURBED_RULE_VIOLATED,DISTURBED_RULE_VIOLATION_COUNT,DISTURBED_RULE_VIOLATION_DURATION,THICK_RULE_VIOLATED,THICK_RULE_VIOLATION_COUNT,THICK_RULE_VIOLATION_DURATION,SMOKE_RULE_VIOLATED,SMOKE_RULE_VIOLATION_COUNT,SMOKE_RULE_VIOLATION_DURATION,FIELD_MILL_RULE_VIOLATED,FIELD_MILL_RULE_VIOLATION_COUNT,FIELD_MILL_RULE_VIOLATION_DURATION,GOOD_SENSE_RULE_VIOLATED,GOOD_SENSE_RULE_VIOLATION_COUNT,GOOD_SENSE_RULE_VIOLATION_DURATION,USER_WX_CONSTRAINT_RULE_VIOLATED,USER_WX_CONSTRAINT_RULE_VIOLATION_COUNT,USER_WX_CONSTRAINT_RULE_VIOLATION_DURATION
175,ATLAS V,SBIRS GEO 6,2022-08-04,2022-08-04 07:29:00,2022-08-04 10:29:00,0 days 03:00:00,0,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00
176,FALCON 9,KPLO,2022-08-04,2022-08-04 21:08:00,2022-08-04 23:08:00,0 days 02:00:00,0,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00
201,FALCON 9,EUTELSAT-10B,2022-11-22,2022-11-22 00:57:00,2022-11-22 01:02:00,0 days 00:05:00,0,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00
202,FALCON 9,CRS-26,2022-11-22,2022-11-22 18:54:00,2022-11-22 20:53:00,0 days 01:59:00,1,False,0,0 days 00:00:00,True,1,0 days 01:59:00,False,0,0 days,True,1,0 days 01:06:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,True,1,0 days 01:59:00,False,0,0 days 00:00:00,False,0,0 days,True,1,0 days 00:06:00,False,0,0 days,True,1,0 days 01:59:00
219,FALCON 9,STARLINK 6-1,2023-02-27,2023-02-27 21:13:00,2023-02-27 23:13:00,0 days 02:00:00,0,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00
220,FALCON 9,CREW-6,2023-02-27,2023-02-27 04:45:00,2023-02-27 06:42:00,0 days 01:57:00,0,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00,False,0,0 days,False,0,0 days 00:00:00


#### Merge Data

In [13]:
# Ensure 'liftoff_date' in launches, 'Date' in wx_decisions and stats are dt
launches['LIFTOFF_DATE'] = pd.to_datetime(launches['LIFTOFF_DATE']).dt.date
stats['DATE'] = pd.to_datetime(stats['DATE']).dt.date
wx_decisions['DATE'] = pd.to_datetime(wx_decisions['DATE']).dt.date

##### Launches x Decisions

In [14]:
# Perform the full outer join of launches and wx_decisions
merged_data = pd.merge(launches, wx_decisions, 
                       left_on='LIFTOFF_DATE', 
                       right_on='DATE', 
                       how='outer', 
                       indicator=True)

# Count successful and unsuccessful merges
successful_merges = (merged_data['_merge'] == 'both').sum()
unsuccessful_merges = (merged_data['_merge'] != 'both').sum()

print(f"Successful merges: {successful_merges}")
print(f"Unsuccessful merges: {unsuccessful_merges}")
print(f"Total rows of merged data: {len(merged_data)}")
display(merged_data.sample(10))

Successful merges: 402
Unsuccessful merges: 503
Total rows of merged data: 905


,NAME,STATUS,PROVIDER,ROCKET,MISSION,LIFTOFF_DATETIME,LOCATION,PAD,LIFTOFF_DATE,LAUNCH_VEHICLE,PAYLOAD,DATE,COUNTDOWN,LAUNCHED,NON_WX_SCRUB,WX_SCRUB,NON_WX_DELAY,WX DELAY,LCC_SCRUB_DELAY,USER_WX_SCRUB_DELAY,REMARKS,_merge
41,Delta 6925-8 | Palapa B2R,Launch Successful,United States Air Force,Delta II,Palapa B2R,1990-04-13 22:28:00,"Cape Canaveral, FL, USA",Space Launch Complex 17B,1990-04-13,DELTA II-194,NaN,1990-04-13,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,both
707,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Pegasus,ICON-3,2019-10-11,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,T-0 delay by 24 minutes due to comms issue,right_only
509,Delta II 7920H-10C | GRAIL-A (Ebb) & GRAIL-B (...,Launch Successful,United Launch Alliance,Delta II,—,2011-09-10 13:08:00,"Cape Canaveral, FL, USA",Space Launch Complex 17B,2011-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
743,Falcon 9 Block 5 | SAOCOM 1B,Launch Successful,SpaceX,Falcon 9,SAOCOM 1B,2020-08-30 23:18:00,"Cape Canaveral, FL, USA",Space Launch Complex 40,2020-08-30,Falcon 9,SAOCOM-1B,2020-08-30,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,both
609,Falcon 9 Full Thrust | Thaicom 8,Launch Successful,SpaceX,Falcon 9,Thaicom 8,2016-05-27 21:39:00,"Cape Canaveral, FL, USA",Space Launch Complex 40,2016-05-27,Falcon 9,THAICOM 8,2016-05-27,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,both
795,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Falcon 9,Inspiration4,2021-09-16,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,right_only
180,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,AC 118,NaN,1995-07-28,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,WIND > 35 KTS,right_only
656,Atlas V 421 | NROL-52,Launch Successful,United Launch Alliance,Atlas V 421,NROL-52,2017-10-15 07:28:00,"Cape Canaveral, FL, USA",Space Launch Complex 41,2017-10-15,ATLAS V,NROL-52,2017-10-15,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,both
101,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,AC-105,NaN,1992-06-27,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,LIGHTNING,right_only
258,Delta II | Skynet 4D,Launch Successful,United States Air Force,Delta II,Skynet 4D,1998-01-10 00:32:00,"Cape Canaveral, FL, USA",Space Launch Complex 17B,1998-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [15]:
print(merged_data[['STATUS', 'PROVIDER', 'LIFTOFF_DATETIME', 'LOCATION', 'PAD','LAUNCH_VEHICLE', 'PAYLOAD', 'DATE', 'COUNTDOWN', 'LAUNCHED', 'WX_SCRUB']].iloc[[465, 564]].transpose().to_markdown(tablefmt='pipe'))

|                  | 465                      | 564                     |
|:-----------------|:-------------------------|:------------------------|
| STATUS           | Launch Successful        | Launch Successful       |
| PROVIDER         | United Launch Alliance   | SpaceX                  |
| LIFTOFF_DATETIME | 2009-01-18 02:47:00      | 2014-09-07 05:00:00     |
| LOCATION         | Cape Canaveral, FL, USA  | Cape Canaveral, FL, USA |
| PAD              | Space Launch Complex 37B | Space Launch Complex 40 |
| LAUNCH_VEHICLE   | nan                      | FALCON 9                |
| PAYLOAD          | nan                      | ASIASAT-6               |
| DATE             | nan                      | 2014-09-07              |
| COUNTDOWN        | nan                      | 1.0                     |
| LAUNCHED         | nan                      | 1.0                     |
| WX_SCRUB         | nan                      | 0.0                     |


##### Merged Launches & Decisions x Stats

In [16]:
# First, drop the _merge column
merged_data = merged_data.drop('_merge', axis=1)

# Join stats to the merged data
merged_data = pd.merge(merged_data, stats,
                       left_on='DATE',
                       right_on='DATE',
                       how='outer',
                       suffixes=('', '_stats'),
                       indicator=True)

# Count successful merges
successful_merges = (merged_data['_merge'] == 'both').sum()
total_merges = len(merged_data)
print(f"Successful merges: {successful_merges} out of {total_merges} total")

Successful merges: 242 out of 914 total


In [17]:
# Ensure boolean columns are kept as int
for col in bool_columns:
    if col in merged_data.columns:
        merged_data[col] = merged_data[col].fillna(0).astype(int)

In [18]:
# Clean up the merge indicator column
merged_data = merged_data.drop('_merge', axis=1)

#### Remove Duplicates and Errors

In [22]:
selected_columns = ['ROCKET', 'MISSION', 'LIFTOFF_DATETIME',
                   'LAUNCH_VEHICLE', 'PAYLOAD', 'DATE', 
                   'LAUNCH_VEHICLE_stats', 'PAYLOAD_stats', 'START_LCC_EVAL', 'END_LCC_EVAL']
duplicate_dates = merged_data[selected_columns][merged_data['DATE'].notna() & merged_data.duplicated(subset='DATE', keep=False)]
display(duplicate_dates)

,ROCKET,MISSION,LIFTOFF_DATETIME,LAUNCH_VEHICLE,PAYLOAD,DATE,LAUNCH_VEHICLE_stats,PAYLOAD_stats,START_LCC_EVAL,END_LCC_EVAL
772,Atlas V 421,SBIRS GEO-6,2022-08-04 10:29:00,Atlas V,SBIRS GEO 6,2022-08-04,ATLAS V,SBIRS GEO 6,2022-08-04 07:29:00,2022-08-04 10:29:00
773,Atlas V 421,SBIRS GEO-6,2022-08-04 10:29:00,Atlas V,SBIRS GEO 6,2022-08-04,FALCON 9,KPLO,2022-08-04 21:08:00,2022-08-04 23:08:00
774,Atlas V 421,SBIRS GEO-6,2022-08-04 10:29:00,Falcon 9,KPLO,2022-08-04,ATLAS V,SBIRS GEO 6,2022-08-04 07:29:00,2022-08-04 10:29:00
775,Atlas V 421,SBIRS GEO-6,2022-08-04 10:29:00,Falcon 9,KPLO,2022-08-04,FALCON 9,KPLO,2022-08-04 21:08:00,2022-08-04 23:08:00
776,Falcon 9,Danuri (KPLO - Korean Pathfinder Lunar Orbiter),2022-08-04 23:08:00,Atlas V,SBIRS GEO 6,2022-08-04,ATLAS V,SBIRS GEO 6,2022-08-04 07:29:00,2022-08-04 10:29:00
777,Falcon 9,Danuri (KPLO - Korean Pathfinder Lunar Orbiter),2022-08-04 23:08:00,Atlas V,SBIRS GEO 6,2022-08-04,FALCON 9,KPLO,2022-08-04 21:08:00,2022-08-04 23:08:00
778,Falcon 9,Danuri (KPLO - Korean Pathfinder Lunar Orbiter),2022-08-04 23:08:00,Falcon 9,KPLO,2022-08-04,ATLAS V,SBIRS GEO 6,2022-08-04 07:29:00,2022-08-04 10:29:00
779,Falcon 9,Danuri (KPLO - Korean Pathfinder Lunar Orbiter),2022-08-04 23:08:00,Falcon 9,KPLO,2022-08-04,FALCON 9,KPLO,2022-08-04 21:08:00,2022-08-04 23:08:00
804,NaN,NaN,NaT,Falcon 9,Eutelsat-10B,2022-11-22,FALCON 9,EUTELSAT-10B,2022-11-22 00:57:00,2022-11-22 01:02:00
805,NaN,NaN,NaT,Falcon 9,Eutelsat-10B,2022-11-22,FALCON 9,CRS-26,2022-11-22 18:54:00,2022-11-22 20:53:00


In [ ]:
# Identify the indexes to remove from merged_data
indexes_to_keep = [772, 779, 804, 807, 824]
indexes_to_remove = duplicate_dates.index[~duplicate_dates.index.isin(indexes_to_keep)]

# Remove the identified rows from merged_data
merged_data = merged_data.drop(indexes_to_remove)

print(f"Removed {len(indexes_to_remove)} rows from merged_data.")
print(f"merged_data now has {len(merged_data)} rows.")

In [ ]:
merged_data[~merged_data['DATE'].isna()]

In [ ]:
print(merged_data[['STATUS', 'PROVIDER', 'LIFTOFF_DATETIME', 'LOCATION', 'PAD', 'LAUNCH_VEHICLE', 'PAYLOAD', 'COUNTDOWN', 'LAUNCHED', 'WX_SCRUB']].loc[[465, 564]].transpose().to_markdown(tablefmt='pipe'))


In [99]:
# merged_data.to_csv('../data/transformed/launch/merged_data.csv', index=False)

## Add Weather Data to Launch Decision Log

In [100]:
# --- Launch Decision Log ---
launch_decision_log = pd.read_csv('../data/transformed/launch/merged_data.csv',
                                  parse_dates=['LIFTOFF_DATETIME', 'LIFTOFF_DATE', 'DATE'],
                                  dtype = {'NAME': str, 'STATUS': str, 'PROVIDER': str, 'ROCKET': str,
                                           'MISSION': str, 'LOCATION': str, 'PAD': str,
                                           'LAUNCH_VEHICLE': str, 'PAYLOAD': str,
                                           'COUNTDOWN': 'Int64', 'LAUNCHED': 'Int64',
                                           'NON_WX_SCRUB': 'Int64', 'WX_SCRUB': 'Int64',
                                           'NON_WX_DELAY': 'Int64', 'WX_DELAY': 'Int64',
                                           'LCC_SCRUB_DELAY': 'Int64', 'USER_WX_SCRUB_DELAY': 'Int64',
                                           'REMARKS': str},
                                    na_values=[' ', 'NaN', 'NA']
                                   )

# --- Hourly Weather Data ---
cape_canaveral_hourly_wx = pd.read_csv('../data/transformed/weather/cape_canaveral_usa_hourly.csv', parse_dates=['time'], dtype={'weather_code (wmo code)': str})
cape_canaveral_hourly_wx.columns = cape_canaveral_hourly_wx.columns.str.upper()

In [ ]:
cape_canaveral_hourly_wx.sample(5)

In [ ]:
# Sample 3 rows and select 8 key variables from cape_canaveral_hourly_wx
sample_wx = cape_canaveral_hourly_wx.sample(3)[['TIME', 'TEMPERATURE_2M (°F)', 'RELATIVE_HUMIDITY_2M (%)', 
                                                'WIND_SPEED_10M (MP/H)', 'WIND_DIRECTION_10M (°)', 
                                                'CLOUD_COVER (%)', 'PRECIPITATION (INCH)', 'WEATHER_CODE (WMO CODE)']]

# Convert the sample to a markdown table
markdown_table = sample_wx.to_markdown(index=False)

# Print the markdown table
print("Sample of Cape Canaveral hourly weather data:")
print(markdown_table)


In [ ]:
nan_locations = launch_decision_log[launch_decision_log['LOCATION'].isna()]  # Show samples where location is NaN
total_entries = len(launch_decision_log)  # Show the percentage of NaN locations
nan_percentage = (len(nan_locations) / total_entries) * 100
print(f"\nTotal number of entries with NaN Location: {len(nan_locations)}")
print(f"Percentage of entries with NaN Location: {nan_percentage:.2f}%")

In [102]:
def get_weather_dataset(location):
    '''
    This function retrieves the weather dataset for a given location.
    If the location is not found in the dataset, it defaults to Cape Canaveral.
    '''
    location_map = {"Cape Canaveral, FL, USA": cape_canaveral_hourly_wx}
    dataset = location_map.get(location)
    if dataset is None:
        dataset = cape_canaveral_hourly_wx

    return dataset

In [103]:
def weighted_average(group):
    weights = np.array([0.02, 0.08, 0.10, 0.80])  # More weight on recent hours
    result = {}
    for col in group.columns:
        if col == 'TIME':
            continue
        elif col == 'WEATHER_CODE (WMO CODE)' or col == 'IS_DAY ()':
            result[col] = most_common_value(group[col])
        else:
            result[col] = np.average(group[col].fillna(0), weights=weights)
    return pd.Series(result)

In [104]:
def most_common_value(series):
    """
    Returns the most common non-null value in a series.
    If all values are null or the series is empty, returns None.
    """
    non_null_values = series.dropna()
    if non_null_values.empty:
        return None
    return Counter(non_null_values).most_common(1)[0][0]

In [105]:
def simple_average(group):
    result = {}
    for col in group.columns:
        if col == 'TIME':
            continue
        elif col == 'WEATHER_CODE (WMO CODE)' or col == 'IS_DAY ()':
            result[col] = most_common_value(group[col])
        else:
            result[col] = group[col].mean()
    return pd.Series(result)

In [106]:
def merge_launch_weather_data(launch_data):
    '''
    This function merges launch data with corresponding weather data for each launch. Here's a breakdown of its functionality:
    1. It iterates through each launch in the provided launch_data DataFrame.
    2. For each launch, it extracts the location and retrieves the corresponding weather dataset.
    3. If no weather data is available for the location, it skips to the next launch.
    4. It then handles two scenarios:
       a. If Liftoff_DateTime is not available but Date is:
          - It uses the entire day's weather data.
          - Calculates a simple average of all weather parameters for that day.
       b. If Liftoff_DateTime is available:
          - It considers the 4 hours preceding the launch.
          - Calculates a weighted average of weather parameters, giving more weight to hours closer to launch time.
    5. If there's insufficient weather data or no valid date/time, it skips the launch.
    6. Finally, it combines the launch data with the averaged weather data and adds it to the result.
    The function returns a new DataFrame containing all launches with their corresponding weather data, where available.`
    '''
    merged_data = []

    for _, launch in launch_data.iterrows():
        location = launch['LOCATION'] if pd.notna(launch['LOCATION']) else None
        weather_data = get_weather_dataset(location)
        
        if weather_data is None:
            print(f"No weather data available for location: {location}")
            continue

        if pd.isna(launch['LIFTOFF_DATETIME']) and pd.notna(launch['DATE']):
            # Use the entire day's data when LIFTOFF_DATETIME is NaT
            launch_date = pd.to_datetime(launch['DATE']).date()
            relevant_weather = weather_data[weather_data['TIME'].dt.date == launch_date]
            
            if len(relevant_weather) == 0:
                print(f"No weather data available for date: {launch_date}")
                continue
            
            avg_weather = simple_average(relevant_weather)
        elif pd.notna(launch['LIFTOFF_DATETIME']):
            # Use the 4 hours preceding the launch when LIFTOFF_DATETIME is available
            launch_time = pd.to_datetime(launch['LIFTOFF_DATETIME'])
            start_time = launch_time - pd.Timedelta(hours=4)
            
            relevant_weather = weather_data[
                (weather_data['TIME'] >= start_time) & 
                (weather_data['TIME'] <= launch_time)
            ].sort_values('TIME').tail(4)

            if len(relevant_weather) < 4:
                print(f"Insufficient weather data for launch at {launch_time}")
                continue

            avg_weather = weighted_average(relevant_weather)
        else:
            print(f"No valid date or time for launch: {launch['NAME']}")
            continue

        merged_row = pd.concat([launch, avg_weather])
        merged_data.append(merged_row)

    result = pd.DataFrame(merged_data)

    return result

In [107]:
# Use the function
# merged_launch_weather = merge_launch_weather_data(launch_decision_log)

In [ ]:
# Save the merged launch and weather data to a CSV file
output_path = '../data/transformed/integrated/launch_corpus.csv'
merged_launch_weather.to_csv(output_path, index=False)
print(f"Merged launch and weather data saved to {output_path}")

### Min Max Approach

In [ ]:
launch_decision_log.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(launch_decision_log['LIFTOFF_DATETIME'].dt.hour, bins=24, color='blue', alpha=0.7)
plt.title('Distribution of Launch Times Throughout the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Launches')
plt.xticks(range(24))
plt.grid(axis='y')
plt.show()

In [ ]:
cape_canaveral_hourly_wx.describe()

In [33]:
def get_relevant_weather_data(weather_data, launch_time=None, date=None, window_before=None, window_after=None):
    """
    Retrieves relevant weather data based on provided time parameters.

    - If launch_time is provided, it will consider window_before and window_after around launch_time.
    - If only date is provided, it will consider the entire day or a specified time window within the day.
    - Returns a DataFrame with the relevant weather data.
    """
    if launch_time is not None:
        # Use the specified window around launch_time
        start_time = launch_time - pd.Timedelta(hours=window_before)
        end_time = launch_time + pd.Timedelta(hours=window_after)
        relevant_weather = weather_data[
            (weather_data['TIME'] >= start_time) &
            (weather_data['TIME'] <= end_time)
        ]
    elif date is not None:
        # Use the specified window within the day or the entire day
        day_start = pd.to_datetime(date)
        if window_before is not None and window_after is not None:
            start_time = day_start + pd.Timedelta(hours=window_before)
            end_time = day_start + pd.Timedelta(hours=window_after)
            relevant_weather = weather_data[
                (weather_data['TIME'] >= start_time) &
                (weather_data['TIME'] <= end_time)
            ]
        else:
            # Use the entire day's data
            relevant_weather = weather_data[weather_data['TIME'].dt.date == day_start.date()]
    else:
        relevant_weather = pd.DataFrame()
    return relevant_weather

In [34]:
def calculate_min_max(group):
    """
    Calculates the min and max for numerical columns in the group.
    Returns a Series with min and max values for each variable.
    """
    result = {}
    for col in group.columns:
        if col == 'TIME':
            continue
        elif col == 'WEATHER_CODE (WMO CODE)' or col == 'IS_DAY ()':
            # For categorical variables, we can use the most common value
            result[col + '_MODE'] = most_common_value(group[col])
        else:
            result[col + '_MIN'] = group[col].min()
            result[col + '_MAX'] = group[col].max()
    return pd.Series(result)

In [36]:
def merge_launch_weather_data_with_min_max(launch_data, window_before_launch=2, window_after_launch=2, default_window_before=None, default_window_after=None):
    '''
    Merges launch data with corresponding weather data, calculating min and max values for each launch.
    This function performs the following steps:

        1. Iterates through each launch in the provided launch_data DataFrame.
        2. Retrieves the weather dataset for the launch location.
        3. Determines the relevant time window for weather data:
            a. If LIFTOFF_DATETIME is available, it uses a window around the launch time.
            b. If only DATE is available, it uses either a specified window or the entire day.
        4. Extracts the relevant weather data for the determined time window.
        5. Calculates minimum and maximum values for numerical weather parameters.
        6. For categorical weather parameters, it determines the most common value.
        7. Combines the launch data with the calculated weather statistics.
        8. Handles various edge cases, such as missing weather data or invalid dates.
        9. Returns a new DataFrame containing merged launch and weather information.
    '''
    merged_data = []

    for _, launch in launch_data.iterrows():
        location = launch['LOCATION'] if pd.notna(launch['LOCATION']) else None
        weather_data = get_weather_dataset(location)
        
        if weather_data is None:
            print(f"No weather data available for location: {location}")
            continue

        if pd.notna(launch['LIFTOFF_DATETIME']):
            # When LIFTOFF_DATETIME is available
            launch_time = pd.to_datetime(launch['LIFTOFF_DATETIME'])
            relevant_weather = get_relevant_weather_data(
                weather_data, 
                launch_time=launch_time, 
                window_before=window_before_launch, 
                window_after=window_after_launch
            )

            if relevant_weather.empty:
                print(f"No weather data available around launch time: {launch_time}")
                continue

        elif pd.notna(launch['DATE']):
            # When only DATE is available
            launch_date = pd.to_datetime(launch['DATE']).date()
            relevant_weather = get_relevant_weather_data(
                weather_data, 
                date=launch_date, 
                window_before=default_window_before, 
                window_after=default_window_after
            )

            if relevant_weather.empty:
                print(f"No weather data available for date: {launch_date}")
                continue
        else:
            print(f"No valid date or time for launch: {launch['NAME']}")
            continue

        # Calculate min and max values
        min_max_weather = calculate_min_max(relevant_weather)

        # Combine launch data with min/max weather data
        merged_row = pd.concat([launch, min_max_weather])
        merged_data.append(merged_row)

    result = pd.DataFrame(merged_data)

    return result

In [ ]:
# Specify the default windows for when LIFTOFF_DATETIME is not available
# For example, to use the entire day, leave default_window_before and default_window_after as None
# To specify a window from 6 AM to 6 PM, set default_window_before=6, default_window_after=18

merged_launch_weather_min_max = merge_launch_weather_data_with_min_max(
    launch_decision_log,
    window_before_launch=2,
    window_after_launch=2,
    default_window_before=None,
    default_window_after=None  # Use None to consider the entire day
)

In [ ]:
merged_launch_weather_min_max.columns.to_list()

In [ ]:
import matplotlib.pyplot as plt

# Filter data where WX_SCRUB is True
scrubbed_weather = merged_launch_weather_min_max[merged_launch_weather_min_max['WX_SCRUB'] == True]

# Filter data where WX_SCRUB is False
non_scrubbed_weather = merged_launch_weather_min_max[merged_launch_weather_min_max['WX_SCRUB'] == False]

# Display the comparison of the two datasets
comparison = pd.concat([scrubbed_weather.head(), non_scrubbed_weather.head()], keys=['WX_SCRUB True', 'WX_SCRUB False'])

# Create graphs for comparison
plt.figure(figsize=(14, 6))

# Plotting temperature comparison
plt.subplot(1, 2, 1)
plt.title('Temperature Comparison')
plt.boxplot([scrubbed_weather['TEMPERATURE_2M (°F)'].dropna(), 
             non_scrubbed_weather['TEMPERATURE_2M (°F)'].dropna()], 
            labels=['WX_SCRUB True', 'WX_SCRUB False'])
plt.ylabel('Temperature (°F)')

# Plotting radiation comparison
plt.subplot(1, 2, 2)
plt.title('Radiation Comparison')
plt.boxplot([scrubbed_weather['TERRESTRIAL_RADIATION_INSTANT (W/M²)'].dropna(), 
             non_scrubbed_weather['TERRESTRIAL_RADIATION_INSTANT (W/M²)'].dropna()], 
            labels=['WX_SCRUB True', 'WX_SCRUB False'])
plt.ylabel('Radiation (W/M²)')

plt.tight_layout()
plt.show()

comparison

In [21]:
merged_launch_weather_min_max.to_csv('../data/transformed/integrated/launch_corpus_min_max.csv', index=False)